In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def add_hours_dam(df, start_hour, interval=48):
    for i in range(start_hour, start_hour+interval):
        start = str(i)
        df["DAM LMP_PRC_"+start+"_hours_ago"] = 0.0
    for i in range(len(df)-1, interval-1, -1):
        for j in range(start_hour, start_hour+interval):
            start = str(j)
            v = df['DAM LMP_PRC'].iloc[i-j,] 
            df.at[i, "DAM LMP_PRC_"+start+"_hours_ago"] =  v
    return df

def add_hours_rtm(df, start_hour, interval=48):
    for rtm in range(1, 13):
        for i in range(start_hour, start_hour+interval):
            start = str(i)
            df["RTM_"+str(rtm) +"_LMP_PRC_"+start+"_hours_ago"] = 0.0
        for i in range(len(df)-1, interval-1, -1):
            for j in range(start_hour, start_hour+interval):
                start = str(j)
                v = df[('RTM LMP_PRC', rtm)].iloc[i-j,] 
                df.at[i, "RTM_"+str(rtm) +"_LMP_PRC_"+start+"_hours_ago"] =  v
    return df

def add_hours_gap(df, start_hour, interval=48):
    for i in range(start_hour, start_hour+interval):
        start = str(i)
        df["GAP LMP_PRC_"+start+"_hours_ago"] = 0.0
    for i in range(len(df)-1, interval-1, -1):
        for j in range(start_hour, start_hour+interval):
            start = str(j)
            v = df['Gap'].iloc[i-j,] 
            df.at[i, "GAP LMP_PRC_"+start+"_hours_ago"] =  v
    return df


In [4]:
feature_names = ['DAM LMP_PRC_1_hours_ago',
 'DAM LMP_PRC_20_hours_ago',
 'Solar Zenith Angle',
 'RTM_10_LMP_PRC_22_hours_ago',
 'DAM LMP_PRC_24_hours_ago',
 'DAM LMP_PRC_19_hours_ago',
 'DAM LMP_PRC_23_hours_ago',
 'RTM_10_LMP_PRC_23_hours_ago',
 'RTM_11_LMP_PRC_21_hours_ago',
 'RTM_8_LMP_PRC_21_hours_ago',
 'Demand_Forecast_Day_Ahead',
 'RTM_9_LMP_PRC_20_hours_ago',
 'RTM_9_LMP_PRC_22_hours_ago',
 'DAM LMP_PRC_21_hours_ago',
 'DAM LMP_PRC_16_hours_ago',
 'RTM_11_LMP_PRC_41_hours_ago',
 'RTM_10_LMP_PRC_21_hours_ago',
 'RTM_9_LMP_PRC_21_hours_ago',
 'RTM_6_LMP_PRC_24_hours_ago',
 'RTM_5_LMP_PRC_20_hours_ago',
 'RTM_10_LMP_PRC_17_hours_ago',
 'hr_index',
 'RTM_8_LMP_PRC_20_hours_ago',
 'RTM_7_LMP_PRC_22_hours_ago',
 'RTM_6_LMP_PRC_21_hours_ago',
 'DAM LMP_PRC_2_hours_ago',
 'Demand_Forecast_2Day_Ahead',
 'RTM_6_LMP_PRC_38_hours_ago',
 'RTM_1_LMP_PRC_41_hours_ago',
 'RTM_4_LMP_PRC_19_hours_ago',
 'RTM_1_LMP_PRC_38_hours_ago',
 'DAM LMP_PRC_22_hours_ago',
 'RTM_6_LMP_PRC_23_hours_ago',
 'RTM_2_LMP_PRC_19_hours_ago',
 'pressure',
 'heat_index_set_1d',
 'RTM_6_LMP_PRC_13_hours_ago',
 'RTM_9_LMP_PRC_28_hours_ago',
 'RTM_6_LMP_PRC_27_hours_ago',
 'RTM_2_LMP_PRC_27_hours_ago',
 'RTM_3_LMP_PRC_33_hours_ago',
 'RTM_8_LMP_PRC_27_hours_ago',
 'DAM LMP_PRC_11_hours_ago',
 'RTM_2_LMP_PRC_38_hours_ago',
 ('DAM RT_Forecast_RTPD', 4),
 'RTM_5_LMP_PRC_58_hours_ago',
 'Clearsky GHI',
 'Hour',
 ('RT_Forecast_RTD', 12),
 'Demand_Forecast_7Day_Ahead',
 'RTM_9_LMP_PRC_32_hours_ago',
 'RTM_10_LMP_PRC_39_hours_ago',
 'RTM_12_LMP_PRC_23_hours_ago',
 'RTM_10_LMP_PRC_56_hours_ago',
 'RTM_9_LMP_PRC_35_hours_ago',
 'RTM_3_LMP_PRC_38_hours_ago',
 'Clearsky DHI',
 'RTM_1_LMP_PRC_20_hours_ago',
 'RTM_1_LMP_PRC_33_hours_ago',
 'RTM_5_LMP_PRC_28_hours_ago',
 'DAM LMP_PRC_3_hours_ago',
 'RTM_5_LMP_PRC_34_hours_ago',
 'RTM_3_LMP_PRC_12_hours_ago',
 'RTM_7_LMP_PRC_15_hours_ago',
 ('RT_Forecast_RTD', 11),
 'RTM_7_LMP_PRC_24_hours_ago',
 'DAM LMP_PRC_12_hours_ago',
 'RTM_11_LMP_PRC_24_hours_ago',
 'RTM_2_LMP_PRC_12_hours_ago',
 'RTM_5_LMP_PRC_23_hours_ago',
 'DAM LMP_PRC_6_hours_ago',
 'RTM_10_LMP_PRC_42_hours_ago',
 'RTM_3_LMP_PRC_59_hours_ago',
 'DAM LMP_PRC_26_hours_ago',
 'RTM_1_LMP_PRC_53_hours_ago',
 'RTM_4_LMP_PRC_12_hours_ago',
 'RTM_3_LMP_PRC_39_hours_ago',
 'RTM_6_LMP_PRC_18_hours_ago',
 'RTM_8_LMP_PRC_14_hours_ago',
 'RTM_3_LMP_PRC_57_hours_ago',
 'RTM_10_LMP_PRC_41_hours_ago',
 ('DAM RT_Forecast_RTPD', 1),
 'RTM_12_LMP_PRC_51_hours_ago',
 'RTM_12_LMP_PRC_24_hours_ago',
 'RTM_9_LMP_PRC_29_hours_ago',
 'altimeter_set_1',
 'RTM_5_LMP_PRC_12_hours_ago',
 ('RT_Forecast_RTD', 10),
 'RTM_2_LMP_PRC_28_hours_ago',
 'DAM LMP_PRC_32_hours_ago',
 'DAM LMP_PRC_7_hours_ago',
 'RTM_9_LMP_PRC_23_hours_ago',
 'RTM_7_LMP_PRC_56_hours_ago',
 ('RT_Forecast_RTD', 7),
 'DAM LMP_PRC_38_hours_ago',
 'Temperature',
 'RTM_7_LMP_PRC_57_hours_ago',
 'RTM_11_LMP_PRC_30_hours_ago',
 'RTM_12_LMP_PRC_25_hours_ago',
 'RTM_1_LMP_PRC_24_hours_ago',
 'RTM_11_LMP_PRC_47_hours_ago',
 'RTM_3_LMP_PRC_21_hours_ago',
 'RTM_4_LMP_PRC_34_hours_ago',
 'RTM_7_LMP_PRC_23_hours_ago',
 'DAM LMP_PRC_8_hours_ago',
 'RTM_12_LMP_PRC_27_hours_ago',
 'RTM_11_LMP_PRC_34_hours_ago',
 'RTM_6_LMP_PRC_58_hours_ago',
 'RTM_9_LMP_PRC_47_hours_ago',
 'DAM LMP_PRC_18_hours_ago',
 'RTM_3_LMP_PRC_37_hours_ago',
 'RTM_1_LMP_PRC_27_hours_ago',
 'RTM_9_LMP_PRC_27_hours_ago',
 'Clearsky DNI',
 ('RT_Forecast_RTD', 1),
 'Wind Direction',
 'RTM_5_LMP_PRC_49_hours_ago',
 'RTM_8_LMP_PRC_12_hours_ago',
 'RTM_3_LMP_PRC_50_hours_ago',
 'Relative Humidity',
 'RTM_5_LMP_PRC_26_hours_ago',
 'Precipitable Water',
 'RTM_11_LMP_PRC_26_hours_ago',
 'RTM_6_LMP_PRC_32_hours_ago',
 'DHI',
 'DNI',
 'RTM_12_LMP_PRC_58_hours_ago',
 'RTM_9_LMP_PRC_58_hours_ago',
 'RTM_10_LMP_PRC_29_hours_ago',
 'DAM LMP_PRC_14_hours_ago',
 'RTM_5_LMP_PRC_33_hours_ago',
 'RTM_1_LMP_PRC_25_hours_ago',
 'DAM LMP_PRC_4_hours_ago',
 'DAM LMP_PRC_9_hours_ago',
 'RTM_12_LMP_PRC_29_hours_ago',
 'RTM_11_LMP_PRC_48_hours_ago',
 'RTM_8_LMP_PRC_43_hours_ago',
 'RTM_6_LMP_PRC_49_hours_ago',
 'DAM LMP_PRC_47_hours_ago',
 'RTM_3_LMP_PRC_18_hours_ago',
 'RTM_10_LMP_PRC_25_hours_ago',
 'RTM_10_LMP_PRC_30_hours_ago',
 'DAM LMP_PRC_46_hours_ago',
 'DAM LMP_PRC_45_hours_ago',
 'RTM_9_LMP_PRC_13_hours_ago',
 'RTM_9_LMP_PRC_25_hours_ago',
 'RTM_2_LMP_PRC_57_hours_ago',
 ('DAM RT_Forecast_RTPD', 3),
 'RTM_8_LMP_PRC_26_hours_ago',
 'RTM_2_LMP_PRC_22_hours_ago',
 'RTM_7_LMP_PRC_34_hours_ago',
 'RTM_8_LMP_PRC_31_hours_ago',
 'RTM_1_LMP_PRC_29_hours_ago',
 'RTM_6_LMP_PRC_55_hours_ago',
 'humidity',
 'RTM_4_LMP_PRC_26_hours_ago',
 'RTM_5_LMP_PRC_51_hours_ago',
 'RTM_4_LMP_PRC_56_hours_ago',
 'RTM_10_LMP_PRC_48_hours_ago',
 'RTM_9_LMP_PRC_49_hours_ago',
 'RTM_10_LMP_PRC_31_hours_ago',
 'RTM_1_LMP_PRC_31_hours_ago',
 'DAM LMP_PRC_15_hours_ago',
 'RTM_8_LMP_PRC_36_hours_ago',
 'RTM_5_LMP_PRC_40_hours_ago',
 'RTM_10_LMP_PRC_24_hours_ago',
 'RTM_11_LMP_PRC_56_hours_ago',
 'relative_humidity_set_1',
 'RTM_8_LMP_PRC_33_hours_ago',
 'RTM_1_LMP_PRC_49_hours_ago',
 'RTM_2_LMP_PRC_14_hours_ago',
 'DAM LMP_PRC_13_hours_ago',
 'RTM_3_LMP_PRC_28_hours_ago',
 'DAM LMP_PRC_27_hours_ago',
 'DAM LMP_PRC_31_hours_ago',
 'RTM_7_LMP_PRC_39_hours_ago',
 'DAM LMP_PRC_25_hours_ago',
 'RTM_11_LMP_PRC_54_hours_ago',
 'RTM_9_LMP_PRC_54_hours_ago',
 'RTM_3_LMP_PRC_35_hours_ago',
 'RTM_8_LMP_PRC_35_hours_ago',
 'RTM_11_LMP_PRC_23_hours_ago',
 'RTM_12_LMP_PRC_15_hours_ago',
 'RTM_1_LMP_PRC_28_hours_ago',
 'RTM_8_LMP_PRC_57_hours_ago',
 'RTM_8_LMP_PRC_23_hours_ago',
 'RTM_5_LMP_PRC_57_hours_ago',
 ('RT_Forecast_RTD', 8),
 'DAM LMP_PRC_10_hours_ago',
 'RTM_4_LMP_PRC_52_hours_ago',
 'RTM_1_LMP_PRC_40_hours_ago',
 'RTM_7_LMP_PRC_31_hours_ago',
 'RTM_11_LMP_PRC_42_hours_ago',
 'RTM_7_LMP_PRC_47_hours_ago',
 'RTM_5_LMP_PRC_17_hours_ago',
 'RTM_3_LMP_PRC_25_hours_ago',
 'GHI',
 'RTM_3_LMP_PRC_24_hours_ago',
 'DAM LMP_PRC_34_hours_ago',
 'RTM_8_LMP_PRC_50_hours_ago',
 'RTM_10_LMP_PRC_32_hours_ago',
 'RTM_12_LMP_PRC_41_hours_ago',
 'RTM_10_LMP_PRC_26_hours_ago',
 'RTM_6_LMP_PRC_25_hours_ago',
 'RTM_12_LMP_PRC_57_hours_ago',
 'sea_level_pressure_set_1d',
 'RTM_9_LMP_PRC_31_hours_ago',
 'DAM LMP_PRC_5_hours_ago',
 'RTM_9_LMP_PRC_24_hours_ago',
 'DAM LMP_PRC_30_hours_ago',
 'RTM_12_LMP_PRC_22_hours_ago',
 ('RT_Forecast_RTD', 9),
 'RTM_12_LMP_PRC_56_hours_ago',
 'RTM_8_LMP_PRC_18_hours_ago',
 'RTM_10_LMP_PRC_19_hours_ago',
 'RTM_5_LMP_PRC_59_hours_ago',
 'RTM_9_LMP_PRC_52_hours_ago',
 'DAM LMP_PRC_28_hours_ago',
 'RTM_6_LMP_PRC_37_hours_ago',
 'RTM_11_LMP_PRC_40_hours_ago',
 'RTM_11_LMP_PRC_38_hours_ago',
 'RTM_4_LMP_PRC_51_hours_ago',
 'DAM LMP_PRC_33_hours_ago',
 'RTM_3_LMP_PRC_51_hours_ago',
 'RTM_10_LMP_PRC_50_hours_ago',
 'RTM_8_LMP_PRC_24_hours_ago',
 'RTM_6_LMP_PRC_40_hours_ago',
 'RTM_11_LMP_PRC_18_hours_ago',
 'RTM_6_LMP_PRC_14_hours_ago',
 'RTM_1_LMP_PRC_47_hours_ago',
 'RTM_6_LMP_PRC_12_hours_ago',
 ('RT_Forecast_RTD', 6),
 'DAM LMP_PRC_43_hours_ago',
 'RTM_3_LMP_PRC_56_hours_ago',
 'RTM_12_LMP_PRC_33_hours_ago',
 'RTM_4_LMP_PRC_50_hours_ago',
 'RTM_7_LMP_PRC_27_hours_ago',
 'DAM LMP_PRC_37_hours_ago',
 'RTM_4_LMP_PRC_25_hours_ago',
 'RTM_1_LMP_PRC_23_hours_ago',
 'DAM LMP_PRC_36_hours_ago',
 'DAM LMP_PRC_39_hours_ago',
 'RTM_11_LMP_PRC_52_hours_ago',
 'RTM_2_LMP_PRC_23_hours_ago',
 'DAM LMP_PRC_48_hours_ago',
 'RTM_12_LMP_PRC_36_hours_ago',
 'RTM_8_LMP_PRC_52_hours_ago',
 'RTM_11_LMP_PRC_55_hours_ago',
 'RTM_1_LMP_PRC_15_hours_ago',
 'RTM_7_LMP_PRC_16_hours_ago',
 'RTM_12_LMP_PRC_45_hours_ago',
 'RTM_2_LMP_PRC_29_hours_ago',
 'RTM_6_LMP_PRC_52_hours_ago',
 'RTM_1_LMP_PRC_22_hours_ago',
 'RTM_3_LMP_PRC_22_hours_ago',
 'RTM_4_LMP_PRC_18_hours_ago',
 'RTM_2_LMP_PRC_56_hours_ago',
 'RTM_4_LMP_PRC_41_hours_ago',
 'RTM_11_LMP_PRC_31_hours_ago',
 'RTM_12_LMP_PRC_30_hours_ago',
 'RTM_5_LMP_PRC_46_hours_ago',
 'RTM_4_LMP_PRC_58_hours_ago',
 'dewpoint',
 'RTM_9_LMP_PRC_34_hours_ago',
 'RTM_1_LMP_PRC_26_hours_ago',
 'RTM_12_LMP_PRC_52_hours_ago',
 'RTM_3_LMP_PRC_47_hours_ago',
 'RTM_2_LMP_PRC_26_hours_ago',
 'RTM_1_LMP_PRC_36_hours_ago',
 'RTM_4_LMP_PRC_31_hours_ago',
 'RTM_12_LMP_PRC_40_hours_ago',
 ('DAM RT_Forecast_RTPD', 2),
 'RTM_8_LMP_PRC_41_hours_ago',
 'RTM_5_LMP_PRC_37_hours_ago',
 'RTM_7_LMP_PRC_30_hours_ago',
 'RTM_12_LMP_PRC_21_hours_ago',
 'RTM_4_LMP_PRC_54_hours_ago',
 'RTM_3_LMP_PRC_55_hours_ago',
 'RTM_4_LMP_PRC_42_hours_ago',
 'RTM_5_LMP_PRC_18_hours_ago',
 'RTM_11_LMP_PRC_14_hours_ago',
 'RTM_8_LMP_PRC_49_hours_ago',
 'RTM_3_LMP_PRC_19_hours_ago',
 'Month',
 'RTM_10_LMP_PRC_58_hours_ago',
 'RTM_10_LMP_PRC_40_hours_ago',
 'RTM_11_LMP_PRC_46_hours_ago',
 'RTM_6_LMP_PRC_26_hours_ago',
 'RTM_3_LMP_PRC_32_hours_ago',
 'RTM_11_LMP_PRC_12_hours_ago',
 'RTM_1_LMP_PRC_12_hours_ago',
 'RTM_12_LMP_PRC_46_hours_ago',
 'RTM_3_LMP_PRC_48_hours_ago',
 'RTM_3_LMP_PRC_23_hours_ago',
 'RTM_10_LMP_PRC_34_hours_ago',
 'Cloud Type',
 'RTM_6_LMP_PRC_33_hours_ago',
 'RTM_2_LMP_PRC_32_hours_ago',
 'RTM_4_LMP_PRC_59_hours_ago',
 'RTM_11_LMP_PRC_33_hours_ago',
 'RTM_1_LMP_PRC_45_hours_ago',
 'RTM_1_LMP_PRC_39_hours_ago',
 'RTM_12_LMP_PRC_59_hours_ago',
 'RTM_6_LMP_PRC_45_hours_ago',
 'RTM_12_LMP_PRC_32_hours_ago',
 'RTM_11_LMP_PRC_22_hours_ago',
 'RTM_6_LMP_PRC_51_hours_ago',
 'RTM_5_LMP_PRC_21_hours_ago',
 'RTM_8_LMP_PRC_39_hours_ago',
 'RTM_12_LMP_PRC_12_hours_ago',
 'RTM_2_LMP_PRC_50_hours_ago',
 'RTM_9_LMP_PRC_50_hours_ago',
 'RTM_11_LMP_PRC_57_hours_ago',
 'RTM_12_LMP_PRC_48_hours_ago',
 'RTM_10_LMP_PRC_43_hours_ago',
 'RTM_9_LMP_PRC_48_hours_ago',
 'RTM_10_LMP_PRC_51_hours_ago',
 'DAM LMP_PRC_44_hours_ago',
 'RTM_5_LMP_PRC_27_hours_ago',
 'DAM LMP_PRC_41_hours_ago',
 'RTM_9_LMP_PRC_42_hours_ago',
 'RTM_2_LMP_PRC_45_hours_ago',
 'RTM_7_LMP_PRC_18_hours_ago',
 'RTM_3_LMP_PRC_49_hours_ago',
 'RTM_4_LMP_PRC_15_hours_ago',
 'RTM_1_LMP_PRC_48_hours_ago',
 'RTM_1_LMP_PRC_19_hours_ago',
 'RTM_4_LMP_PRC_39_hours_ago',
 'RTM_10_LMP_PRC_49_hours_ago',
 'RTM_5_LMP_PRC_47_hours_ago',
 'RTM_5_LMP_PRC_45_hours_ago',
 'RTM_2_LMP_PRC_49_hours_ago',
 'RTM_5_LMP_PRC_42_hours_ago',
 'RTM_11_LMP_PRC_20_hours_ago',
 'RTM_1_LMP_PRC_57_hours_ago',
 'RTM_8_LMP_PRC_59_hours_ago',
 'RTM_7_LMP_PRC_50_hours_ago',
 'RTM_6_LMP_PRC_31_hours_ago',
 'RTM_5_LMP_PRC_13_hours_ago',
 'weather_condition_set_1d',
 'RTM_5_LMP_PRC_24_hours_ago',
 'RTM_7_LMP_PRC_51_hours_ago',
 'RTM_4_LMP_PRC_33_hours_ago',
 'RTM_12_LMP_PRC_20_hours_ago',
 'RTM_5_LMP_PRC_22_hours_ago',
 'RTM_1_LMP_PRC_55_hours_ago',
 'RTM_1_LMP_PRC_46_hours_ago',
 'RTM_10_LMP_PRC_15_hours_ago',
 'RTM_4_LMP_PRC_45_hours_ago',
 'RTM_3_LMP_PRC_27_hours_ago',
 'RTM_8_LMP_PRC_58_hours_ago',
 'RTM_6_LMP_PRC_42_hours_ago',
 'RTM_4_LMP_PRC_30_hours_ago',
 'RTM_12_LMP_PRC_26_hours_ago',
 'RTM_5_LMP_PRC_55_hours_ago',
 'RTM_2_LMP_PRC_21_hours_ago',
 'RTM_2_LMP_PRC_44_hours_ago',
 'RTM_2_LMP_PRC_15_hours_ago',
 ('RT_Forecast_RTD', 5),
 'RTM_3_LMP_PRC_41_hours_ago',
 'RTM_6_LMP_PRC_50_hours_ago',
 'RTM_2_LMP_PRC_20_hours_ago',
 'RTM_5_LMP_PRC_19_hours_ago',
 'RTM_11_LMP_PRC_25_hours_ago',
 'RTM_10_LMP_PRC_20_hours_ago',
 'RTM_4_LMP_PRC_17_hours_ago',
 'RTM_9_LMP_PRC_56_hours_ago',
 'RTM_9_LMP_PRC_51_hours_ago',
 'DAM LMP_PRC_29_hours_ago',
 'RTM_9_LMP_PRC_33_hours_ago',
 'RTM_9_LMP_PRC_39_hours_ago',
 'RTM_9_LMP_PRC_57_hours_ago',
 'RTM_10_LMP_PRC_12_hours_ago',
 'RTM_6_LMP_PRC_44_hours_ago',
 'RTM_4_LMP_PRC_46_hours_ago',
 'RTM_10_LMP_PRC_53_hours_ago',
 'RTM_2_LMP_PRC_25_hours_ago',
 'RTM_4_LMP_PRC_24_hours_ago',
 'RTM_1_LMP_PRC_30_hours_ago',
 'RTM_2_LMP_PRC_37_hours_ago',
 'RTM_5_LMP_PRC_48_hours_ago',
 'RTM_3_LMP_PRC_58_hours_ago',
 'RTM_4_LMP_PRC_21_hours_ago',
 'RTM_10_LMP_PRC_14_hours_ago',
 'RTM_3_LMP_PRC_42_hours_ago',
 'RTM_2_LMP_PRC_33_hours_ago',
 'RTM_8_LMP_PRC_19_hours_ago',
 'RTM_8_LMP_PRC_22_hours_ago',
 'RTM_11_LMP_PRC_51_hours_ago',
 'RTM_2_LMP_PRC_43_hours_ago',
 'cloud_layer_3_code_set_1',
 'RTM_1_LMP_PRC_50_hours_ago',
 'RTM_7_LMP_PRC_32_hours_ago',
 'RTM_8_LMP_PRC_30_hours_ago',
 'RTM_9_LMP_PRC_41_hours_ago',
 'RTM_6_LMP_PRC_30_hours_ago',
 'RTM_8_LMP_PRC_17_hours_ago',
 'RTM_7_LMP_PRC_12_hours_ago',
 'RTM_3_LMP_PRC_31_hours_ago',
 'RTM_10_LMP_PRC_27_hours_ago',
 'RTM_7_LMP_PRC_46_hours_ago',
 'RTM_9_LMP_PRC_14_hours_ago',
 ('RT_Forecast_RTD', 4),
 'RTM_7_LMP_PRC_49_hours_ago',
 'RTM_3_LMP_PRC_20_hours_ago',
 'RTM_7_LMP_PRC_37_hours_ago',
 'RTM_11_LMP_PRC_58_hours_ago',
 'RTM_7_LMP_PRC_38_hours_ago',
 'RTM_4_LMP_PRC_27_hours_ago',
 'RTM_9_LMP_PRC_55_hours_ago',
 'RTM_11_LMP_PRC_37_hours_ago',
 'RTM_10_LMP_PRC_55_hours_ago',
 'RTM_4_LMP_PRC_44_hours_ago',
 'RTM_5_LMP_PRC_38_hours_ago',
 'RTM_1_LMP_PRC_44_hours_ago',
 'RTM_3_LMP_PRC_46_hours_ago',
 'RTM_11_LMP_PRC_50_hours_ago',
 'RTM_7_LMP_PRC_42_hours_ago',
 'RTM_2_LMP_PRC_54_hours_ago',
 'RTM_1_LMP_PRC_21_hours_ago',
 'RTM_6_LMP_PRC_53_hours_ago',
 'RTM_2_LMP_PRC_30_hours_ago',
 'RTM_1_LMP_PRC_18_hours_ago',
 'DAM LMP_PRC_17_hours_ago',
 'RTM_3_LMP_PRC_26_hours_ago',
 'DAM LMP_PRC_42_hours_ago',
 'RTM_1_LMP_PRC_16_hours_ago',
 'RTM_6_LMP_PRC_48_hours_ago',
 'RTM_2_LMP_PRC_51_hours_ago',
 'RTM_12_LMP_PRC_43_hours_ago',
 'RTM_6_LMP_PRC_20_hours_ago',
 'RTM_9_LMP_PRC_40_hours_ago',
 'RTM_7_LMP_PRC_26_hours_ago',
 'RTM_2_LMP_PRC_24_hours_ago',
 'RTM_2_LMP_PRC_53_hours_ago',
 'RTM_1_LMP_PRC_56_hours_ago',
 'DAM LMP_PRC_40_hours_ago',
 'RTM_3_LMP_PRC_36_hours_ago',
 'RTM_4_LMP_PRC_23_hours_ago',
 'RTM_3_LMP_PRC_45_hours_ago',
 'RTM_9_LMP_PRC_17_hours_ago',
 'RTM_10_LMP_PRC_28_hours_ago',
 'RTM_7_LMP_PRC_20_hours_ago',
 'RTM_2_LMP_PRC_48_hours_ago',
 'RTM_10_LMP_PRC_18_hours_ago',
 'RTM_4_LMP_PRC_40_hours_ago',
 'RTM_8_LMP_PRC_48_hours_ago',
 'RTM_10_LMP_PRC_36_hours_ago',
 'RTM_11_LMP_PRC_17_hours_ago',
 'RTM_3_LMP_PRC_14_hours_ago',
 'RTM_7_LMP_PRC_43_hours_ago',
 'RTM_7_LMP_PRC_44_hours_ago',
 'RTM_6_LMP_PRC_15_hours_ago',
 'RTM_12_LMP_PRC_44_hours_ago',
 'RTM_12_LMP_PRC_47_hours_ago',
 'RTM_1_LMP_PRC_34_hours_ago',
 'RTM_7_LMP_PRC_55_hours_ago',
 'RTM_3_LMP_PRC_53_hours_ago',
 'RTM_8_LMP_PRC_42_hours_ago',
 'RTM_6_LMP_PRC_43_hours_ago',
 'RTM_6_LMP_PRC_19_hours_ago',
 'RTM_5_LMP_PRC_32_hours_ago',
 'RTM_6_LMP_PRC_57_hours_ago',
 'RTM_9_LMP_PRC_15_hours_ago',
 ('RT_Forecast_RTD', 2),
 'RTM_2_LMP_PRC_40_hours_ago',
 'RTM_10_LMP_PRC_13_hours_ago',
 'RTM_4_LMP_PRC_49_hours_ago',
 'RTM_2_LMP_PRC_35_hours_ago',
 'RTM_9_LMP_PRC_38_hours_ago',
 'RTM_2_LMP_PRC_42_hours_ago',
 'RTM_9_LMP_PRC_30_hours_ago',
 'RTM_6_LMP_PRC_29_hours_ago',
 'RTM_8_LMP_PRC_46_hours_ago',
 'RTM_8_LMP_PRC_28_hours_ago',
 'RTM_12_LMP_PRC_49_hours_ago',
 'RTM_3_LMP_PRC_54_hours_ago',
 'RTM_5_LMP_PRC_35_hours_ago',
 'RTM_2_LMP_PRC_13_hours_ago',
 'RTM_4_LMP_PRC_38_hours_ago',
 'RTM_9_LMP_PRC_53_hours_ago',
 'RTM_10_LMP_PRC_45_hours_ago',
 'RTM_8_LMP_PRC_56_hours_ago',
 'RTM_8_LMP_PRC_53_hours_ago',
 'RTM_9_LMP_PRC_19_hours_ago',
 'RTM_12_LMP_PRC_16_hours_ago',
 'RTM_3_LMP_PRC_43_hours_ago',
 'RTM_7_LMP_PRC_29_hours_ago',
 'DAM LMP_PRC_35_hours_ago',
 'RTM_12_LMP_PRC_35_hours_ago',
 'RTM_8_LMP_PRC_29_hours_ago',
 'RTM_6_LMP_PRC_28_hours_ago',
 'RTM_7_LMP_PRC_13_hours_ago',
 'RTM_9_LMP_PRC_44_hours_ago',
 'RTM_1_LMP_PRC_52_hours_ago',
 'RTM_7_LMP_PRC_45_hours_ago',
 'RTM_11_LMP_PRC_16_hours_ago',
 'RTM_11_LMP_PRC_53_hours_ago',
 'RTM_5_LMP_PRC_36_hours_ago',
 'RTM_6_LMP_PRC_59_hours_ago',
 'RTM_12_LMP_PRC_31_hours_ago',
 'RTM_1_LMP_PRC_32_hours_ago',
 'RTM_4_LMP_PRC_55_hours_ago',
 'RTM_3_LMP_PRC_44_hours_ago',
 'RTM_6_LMP_PRC_41_hours_ago',
 'RTM_7_LMP_PRC_58_hours_ago',
 'RTM_7_LMP_PRC_40_hours_ago',
 'RTM_2_LMP_PRC_52_hours_ago',
 'RTM_7_LMP_PRC_53_hours_ago',
 'RTM_4_LMP_PRC_29_hours_ago',
 'RTM_9_LMP_PRC_18_hours_ago',
 'RTM_12_LMP_PRC_38_hours_ago',
 'RTM_8_LMP_PRC_15_hours_ago',
 'RTM_8_LMP_PRC_13_hours_ago',
 'RTM_4_LMP_PRC_22_hours_ago',
 'RTM_6_LMP_PRC_47_hours_ago',
 'RTM_12_LMP_PRC_39_hours_ago',
 'RTM_11_LMP_PRC_39_hours_ago',
 'RTM_5_LMP_PRC_25_hours_ago',
 'RTM_7_LMP_PRC_17_hours_ago',
 'RTM_8_LMP_PRC_40_hours_ago',
 'RTM_11_LMP_PRC_32_hours_ago',
 'RTM_11_LMP_PRC_13_hours_ago',
 'RTM_10_LMP_PRC_37_hours_ago',
 'RTM_8_LMP_PRC_16_hours_ago',
 'RTM_12_LMP_PRC_17_hours_ago',
 'RTM_2_LMP_PRC_46_hours_ago',
 'RTM_3_LMP_PRC_30_hours_ago',
 'RTM_5_LMP_PRC_41_hours_ago',
 'RTM_1_LMP_PRC_37_hours_ago',
 'RTM_10_LMP_PRC_47_hours_ago',
 'RTM_11_LMP_PRC_19_hours_ago',
 'RTM_2_LMP_PRC_41_hours_ago',
 'RTM_8_LMP_PRC_54_hours_ago',
 'RTM_8_LMP_PRC_25_hours_ago',
 'RTM_5_LMP_PRC_43_hours_ago',
 'RTM_4_LMP_PRC_48_hours_ago',
 'RTM_12_LMP_PRC_28_hours_ago',
 'RTM_12_LMP_PRC_13_hours_ago',
 'RTM_1_LMP_PRC_59_hours_ago',
 'cloud_layer_1_code_set_1',
 'RTM_7_LMP_PRC_14_hours_ago',
 'RTM_7_LMP_PRC_19_hours_ago',
 'RTM_6_LMP_PRC_36_hours_ago',
 'RTM_4_LMP_PRC_13_hours_ago',
 'RTM_12_LMP_PRC_53_hours_ago',
 'RTM_5_LMP_PRC_14_hours_ago',
 'RTM_4_LMP_PRC_14_hours_ago',
 'RTM_5_LMP_PRC_54_hours_ago',
 'RTM_3_LMP_PRC_15_hours_ago',
 'RTM_8_LMP_PRC_55_hours_ago',
 'RTM_6_LMP_PRC_54_hours_ago',
 ('RT_Forecast_RTD', 3),
 'RTM_4_LMP_PRC_37_hours_ago',
 'RTM_2_LMP_PRC_36_hours_ago',
 'RTM_11_LMP_PRC_44_hours_ago',
 'RTM_5_LMP_PRC_50_hours_ago',
 'RTM_4_LMP_PRC_35_hours_ago',
 'RTM_12_LMP_PRC_54_hours_ago',
 'RTM_12_LMP_PRC_18_hours_ago',
 'RTM_9_LMP_PRC_59_hours_ago',
 'RTM_10_LMP_PRC_46_hours_ago',
 'RTM_2_LMP_PRC_59_hours_ago',
 'RTM_1_LMP_PRC_13_hours_ago',
 'RTM_6_LMP_PRC_46_hours_ago',
 'RTM_1_LMP_PRC_58_hours_ago',
 'windspeed',
 'RTM_11_LMP_PRC_45_hours_ago',
 'RTM_5_LMP_PRC_53_hours_ago',
 'RTM_7_LMP_PRC_59_hours_ago',
 'RTM_11_LMP_PRC_43_hours_ago',
 'RTM_2_LMP_PRC_58_hours_ago',
 'RTM_10_LMP_PRC_38_hours_ago',
 'RTM_3_LMP_PRC_29_hours_ago',
 'RTM_12_LMP_PRC_42_hours_ago',
 'RTM_9_LMP_PRC_43_hours_ago',
 'RTM_2_LMP_PRC_31_hours_ago',
 'RTM_3_LMP_PRC_34_hours_ago',
 'RTM_1_LMP_PRC_54_hours_ago',
 'RTM_7_LMP_PRC_48_hours_ago',
 'RTM_7_LMP_PRC_25_hours_ago',
 'RTM_9_LMP_PRC_46_hours_ago',
 'RTM_3_LMP_PRC_40_hours_ago',
 'RTM_3_LMP_PRC_17_hours_ago',
 'RTM_2_LMP_PRC_47_hours_ago',
 'RTM_7_LMP_PRC_21_hours_ago',
 'RTM_5_LMP_PRC_31_hours_ago',
 'RTM_8_LMP_PRC_44_hours_ago',
 'RTM_4_LMP_PRC_20_hours_ago',
 'RTM_2_LMP_PRC_16_hours_ago',
 'RTM_3_LMP_PRC_16_hours_ago',
 'RTM_11_LMP_PRC_49_hours_ago',
 'RTM_8_LMP_PRC_51_hours_ago',
 'RTM_11_LMP_PRC_36_hours_ago',
 'RTM_2_LMP_PRC_17_hours_ago',
 'RTM_2_LMP_PRC_34_hours_ago',
 'RTM_5_LMP_PRC_52_hours_ago',
 'RTM_1_LMP_PRC_42_hours_ago',
 'ceiling_set_1',
 'RTM_12_LMP_PRC_37_hours_ago',
 'RTM_11_LMP_PRC_59_hours_ago',
 'RTM_5_LMP_PRC_15_hours_ago',
 'RTM_10_LMP_PRC_44_hours_ago',
 'RTM_4_LMP_PRC_47_hours_ago',
 'RTM_9_LMP_PRC_26_hours_ago',
 'RTM_11_LMP_PRC_27_hours_ago',
 'RTM_7_LMP_PRC_36_hours_ago',
 'RTM_5_LMP_PRC_16_hours_ago',
 'RTM_8_LMP_PRC_47_hours_ago',
 'RTM_4_LMP_PRC_28_hours_ago',
 'RTM_5_LMP_PRC_39_hours_ago',
 'Day',
 'RTM_2_LMP_PRC_55_hours_ago',
 'RTM_5_LMP_PRC_30_hours_ago',
 'Year',
 'RTM_8_LMP_PRC_32_hours_ago',
 'RTM_9_LMP_PRC_12_hours_ago',
 'RTM_6_LMP_PRC_56_hours_ago',
 'RTM_10_LMP_PRC_33_hours_ago',
 'RTM_3_LMP_PRC_52_hours_ago',
 'RTM_11_LMP_PRC_35_hours_ago',
 'RTM_12_LMP_PRC_50_hours_ago',
 'RTM_4_LMP_PRC_53_hours_ago',
 'RTM_6_LMP_PRC_17_hours_ago',
 'RTM_11_LMP_PRC_29_hours_ago',
 'RTM_1_LMP_PRC_17_hours_ago',
 'RTM_7_LMP_PRC_52_hours_ago',
 'RTM_8_LMP_PRC_38_hours_ago',
 'RTM_9_LMP_PRC_45_hours_ago',
 'RTM_6_LMP_PRC_22_hours_ago',
 'RTM_4_LMP_PRC_36_hours_ago',
 'RTM_1_LMP_PRC_51_hours_ago',
 'RTM_11_LMP_PRC_28_hours_ago',
 'RTM_3_LMP_PRC_13_hours_ago',
 'RTM_11_LMP_PRC_15_hours_ago',
 'Fill Flag',
 'RTM_4_LMP_PRC_32_hours_ago',
 'RTM_1_LMP_PRC_43_hours_ago',
 'RTM_10_LMP_PRC_52_hours_ago',
 'RTM_1_LMP_PRC_14_hours_ago',
 'RTM_7_LMP_PRC_33_hours_ago',
 'RTM_4_LMP_PRC_43_hours_ago',
 'visibility_set_1',
 'RTM_10_LMP_PRC_35_hours_ago',
 'RTM_10_LMP_PRC_57_hours_ago',
 'RTM_12_LMP_PRC_55_hours_ago',
 'RTM_4_LMP_PRC_16_hours_ago',
 'RTM_12_LMP_PRC_14_hours_ago',
 'RTM_1_LMP_PRC_35_hours_ago',
 'RTM_10_LMP_PRC_59_hours_ago',
 'RTM_8_LMP_PRC_34_hours_ago',
 'RTM_9_LMP_PRC_16_hours_ago',
 'RTM_7_LMP_PRC_35_hours_ago',
 'RTM_6_LMP_PRC_35_hours_ago',
 'RTM_4_LMP_PRC_57_hours_ago',
 'RTM_10_LMP_PRC_54_hours_ago',
 'RTM_6_LMP_PRC_16_hours_ago',
 'RTM_12_LMP_PRC_34_hours_ago',
 'RTM_8_LMP_PRC_37_hours_ago',
 'RTM_12_LMP_PRC_19_hours_ago',
 'RTM_5_LMP_PRC_29_hours_ago',
 'RTM_10_LMP_PRC_16_hours_ago',
 'RTM_9_LMP_PRC_37_hours_ago',
 'cloud_layer_2_code_set_1',
 'RTM_2_LMP_PRC_39_hours_ago',
 'RTM_8_LMP_PRC_45_hours_ago',
 'RTM_6_LMP_PRC_39_hours_ago',
 'RTM_5_LMP_PRC_56_hours_ago',
 'RTM_7_LMP_PRC_54_hours_ago',
 'weather_cond_code_set_1',
 'IsWeekend',
 'RTM_7_LMP_PRC_28_hours_ago',
 'RTM_2_LMP_PRC_18_hours_ago',
 'RTM_6_LMP_PRC_34_hours_ago',
 'RTM_7_LMP_PRC_41_hours_ago',
 'RTM_9_LMP_PRC_36_hours_ago',
 'RTM_5_LMP_PRC_44_hours_ago',
 'IsHoliday',
 'IsWeekday',
 'wind_gust_set_1',
 'precip_accum_one_hour_set_1',
 'wind_chill_set_1d',
 'precipitation',
 'precipitation_6']


In [23]:
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

f_predictions = []


df = pd.read_pickle('df.pkl')
df = df.reset_index(drop=True)
df = add_hours_dam(df, 1)
df = add_hours_rtm(df, 12)
Y_cols = ['DAM LMP_PRC'] 
X_cols = [col for col in feature_names if col not in Y_cols+[
('HASP LMP_CONG_PRC', 1),
('HASP LMP_CONG_PRC', 2),
('HASP LMP_CONG_PRC', 3),
('HASP LMP_CONG_PRC', 4),
('HASP LMP_ENE_PRC', 1),
('HASP LMP_ENE_PRC', 2),
('HASP LMP_ENE_PRC', 3),
('HASP LMP_ENE_PRC', 4),
('HASP LMP_GHG_PRC', 1),
('HASP LMP_LOSS_PRC', 1),
('HASP LMP_LOSS_PRC', 2),
('HASP LMP_LOSS_PRC', 3),
('HASP LMP_LOSS_PRC', 4),
('HASP LMP_PRC', 1),
('HASP LMP_PRC', 2),
('HASP LMP_PRC', 3),
('HASP LMP_PRC', 4),
('RTM LMP_CONG_PRC', 1),
('RTM LMP_CONG_PRC', 2),
('RTM LMP_CONG_PRC', 3),
('RTM LMP_CONG_PRC', 4),
('RTM LMP_CONG_PRC', 5),
('RTM LMP_CONG_PRC', 6),
('RTM LMP_CONG_PRC', 7),
('RTM LMP_CONG_PRC', 8),
('RTM LMP_CONG_PRC', 9),
('RTM LMP_CONG_PRC', 10),
('RTM LMP_CONG_PRC', 11),
('RTM LMP_CONG_PRC', 12),
('RTM LMP_ENE_PRC', 1),
('RTM LMP_ENE_PRC', 2),
('RTM LMP_ENE_PRC', 3),
('RTM LMP_ENE_PRC', 4),
('RTM LMP_ENE_PRC', 5),
('RTM LMP_ENE_PRC', 6),
('RTM LMP_ENE_PRC', 7),
('RTM LMP_ENE_PRC', 8),
('RTM LMP_ENE_PRC', 9),
('RTM LMP_ENE_PRC', 10),
('RTM LMP_ENE_PRC', 11),
('RTM LMP_ENE_PRC', 12),
('RTM LMP_GHG_PRC', 1),
('RTM LMP_GHG_PRC', 7),
('RTM LMP_LOSS_PRC', 1),
('RTM LMP_LOSS_PRC', 2),
('RTM LMP_LOSS_PRC', 3),
('RTM LMP_LOSS_PRC', 4),
('RTM LMP_LOSS_PRC', 5),
('RTM LMP_LOSS_PRC', 6),
('RTM LMP_LOSS_PRC', 7),
('RTM LMP_LOSS_PRC', 8),
('RTM LMP_LOSS_PRC', 9),
('RTM LMP_LOSS_PRC', 10),
('RTM LMP_LOSS_PRC', 11),
('RTM LMP_LOSS_PRC', 12),
('RTM LMP_PRC', 1),
('TM LMP_PRC', 2),
('RTM LMP_PRC', 3),
('RTM LMP_PRC', 4),
('RTM LMP_PRC', 5),
('RTM LMP_PRC', 6),
('RTM LMP_PRC', 7),
('RTM LMP_PRC', 8),
('RTM LMP_PRC', 9),
('RTM LMP_PRC', 10),
('RTM LMP_PRC', 11),
('RTM LMP_PRC', 12),
'DAM LMP_CONG_PRC',
'DAM LMP_GHG_PRC',
('HASP LMP_CONG_PRC', 1),
('HASP LMP_CONG_PRC', 2),
('HASP LMP_CONG_PRC', 3),
('HASP LMP_CONG_PRC', 4),
('HASP LMP_ENE_PRC', 1),
('HASP LMP_ENE_PRC', 2),
('HASP LMP_ENE_PRC', 3),
('HASP LMP_ENE_PRC', 4),
('HASP LMP_GHG_PRC', 1),
('HASP LMP_LOSS_PRC', 1),
('HASP LMP_LOSS_PRC', 2),
('HASP LMP_LOSS_PRC', 3),
('HASP LMP_LOSS_PRC', 4),
('HASP LMP_PRC', 1),
('HASP LMP_PRC', 2),
('HASP LMP_PRC', 3),
('HASP LMP_PRC', 4),
('RTM LMP_CONG_PRC', 1),
('RTM LMP_CONG_PRC', 2),
('RTM LMP_CONG_PRC', 3),
('RTM LMP_CONG_PRC', 4),
('RTM LMP_CONG_PRC', 5),
('RTM LMP_CONG_PRC', 6),
('RTM LMP_CONG_PRC', 7),
('RTM LMP_CONG_PRC', 8),
('RTM LMP_CONG_PRC', 9),
('RTM LMP_CONG_PRC', 10),
('RTM LMP_CONG_PRC', 11),
('RTM LMP_CONG_PRC', 12),
('RTM LMP_GHG_PRC', 1),
('RTM LMP_GHG_PRC', 7),
'DAM LMP_ENE_PRC',
'DAM LMP_LOSS_PRC',
'Actual_Demand',
('RTM LMP_PRC', 1),
'DAM LMP_PRC',]]
          
Y = df[Y_cols]
X = df[X_cols]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10615, shuffle=False, random_state=42)

ss1 = MinMaxScaler(feature_range=(0, 1))
X_train = ss1.fit_transform(X_train)
X_test = ss1.transform(X_test)
ss2 = MinMaxScaler(feature_range=(0, 1))
Y_train = ss2.fit_transform(Y_train)
Y_test = ss2.transform(Y_test)



from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 100, num = 2)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 2)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


# # Use the random grid to search for best hyperparameters
# # First create the base model to tune
rf = RandomForestRegressor()
# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
rf_random = GridSearchCV(estimator = rf, param_grid = random_grid,scoring= 'neg_mean_squared_error', 
                        cv = 5, 
                        return_train_score=True,
                        verbose = 1,
                        n_jobs=-1)
# # Fit the random search model
rf_random.fit(X_train, Y_train)



Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 17.9min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 74.4min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 225.2min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 237.8min finished
C:\Users\Nika\Anaconda3\envs\PricePrediction\lib\site-packages\sklearn\model_selection\_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [ ]:
rf_random.best_params_

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

f_predictions = []


df = pd.read_pickle('df.pkl')
df = df.reset_index(drop=True)
df = add_hours_dam(df, 1)
df = add_hours_rtm(df, 12)
Y_cols = ['DAM LMP_PRC'] 
X_cols = [col for col in feature_names if col not in Y_cols+[
('HASP LMP_CONG_PRC', 1),
('HASP LMP_CONG_PRC', 2),
('HASP LMP_CONG_PRC', 3),
('HASP LMP_CONG_PRC', 4),
('HASP LMP_ENE_PRC', 1),
('HASP LMP_ENE_PRC', 2),
('HASP LMP_ENE_PRC', 3),
('HASP LMP_ENE_PRC', 4),
('HASP LMP_GHG_PRC', 1),
('HASP LMP_LOSS_PRC', 1),
('HASP LMP_LOSS_PRC', 2),
('HASP LMP_LOSS_PRC', 3),
('HASP LMP_LOSS_PRC', 4),
('HASP LMP_PRC', 1),
('HASP LMP_PRC', 2),
('HASP LMP_PRC', 3),
('HASP LMP_PRC', 4),
('RTM LMP_CONG_PRC', 1),
('RTM LMP_CONG_PRC', 2),
('RTM LMP_CONG_PRC', 3),
('RTM LMP_CONG_PRC', 4),
('RTM LMP_CONG_PRC', 5),
('RTM LMP_CONG_PRC', 6),
('RTM LMP_CONG_PRC', 7),
('RTM LMP_CONG_PRC', 8),
('RTM LMP_CONG_PRC', 9),
('RTM LMP_CONG_PRC', 10),
('RTM LMP_CONG_PRC', 11),
('RTM LMP_CONG_PRC', 12),
('RTM LMP_ENE_PRC', 1),
('RTM LMP_ENE_PRC', 2),
('RTM LMP_ENE_PRC', 3),
('RTM LMP_ENE_PRC', 4),
('RTM LMP_ENE_PRC', 5),
('RTM LMP_ENE_PRC', 6),
('RTM LMP_ENE_PRC', 7),
('RTM LMP_ENE_PRC', 8),
('RTM LMP_ENE_PRC', 9),
('RTM LMP_ENE_PRC', 10),
('RTM LMP_ENE_PRC', 11),
('RTM LMP_ENE_PRC', 12),
('RTM LMP_GHG_PRC', 1),
('RTM LMP_GHG_PRC', 7),
('RTM LMP_LOSS_PRC', 1),
('RTM LMP_LOSS_PRC', 2),
('RTM LMP_LOSS_PRC', 3),
('RTM LMP_LOSS_PRC', 4),
('RTM LMP_LOSS_PRC', 5),
('RTM LMP_LOSS_PRC', 6),
('RTM LMP_LOSS_PRC', 7),
('RTM LMP_LOSS_PRC', 8),
('RTM LMP_LOSS_PRC', 9),
('RTM LMP_LOSS_PRC', 10),
('RTM LMP_LOSS_PRC', 11),
('RTM LMP_LOSS_PRC', 12),
('RTM LMP_PRC', 1),
('TM LMP_PRC', 2),
('RTM LMP_PRC', 3),
('RTM LMP_PRC', 4),
('RTM LMP_PRC', 5),
('RTM LMP_PRC', 6),
('RTM LMP_PRC', 7),
('RTM LMP_PRC', 8),
('RTM LMP_PRC', 9),
('RTM LMP_PRC', 10),
('RTM LMP_PRC', 11),
('RTM LMP_PRC', 12),
'DAM LMP_CONG_PRC',
'DAM LMP_GHG_PRC',
('HASP LMP_CONG_PRC', 1),
('HASP LMP_CONG_PRC', 2),
('HASP LMP_CONG_PRC', 3),
('HASP LMP_CONG_PRC', 4),
('HASP LMP_ENE_PRC', 1),
('HASP LMP_ENE_PRC', 2),
('HASP LMP_ENE_PRC', 3),
('HASP LMP_ENE_PRC', 4),
('HASP LMP_GHG_PRC', 1),
('HASP LMP_LOSS_PRC', 1),
('HASP LMP_LOSS_PRC', 2),
('HASP LMP_LOSS_PRC', 3),
('HASP LMP_LOSS_PRC', 4),
('HASP LMP_PRC', 1),
('HASP LMP_PRC', 2),
('HASP LMP_PRC', 3),
('HASP LMP_PRC', 4),
('RTM LMP_CONG_PRC', 1),
('RTM LMP_CONG_PRC', 2),
('RTM LMP_CONG_PRC', 3),
('RTM LMP_CONG_PRC', 4),
('RTM LMP_CONG_PRC', 5),
('RTM LMP_CONG_PRC', 6),
('RTM LMP_CONG_PRC', 7),
('RTM LMP_CONG_PRC', 8),
('RTM LMP_CONG_PRC', 9),
('RTM LMP_CONG_PRC', 10),
('RTM LMP_CONG_PRC', 11),
('RTM LMP_CONG_PRC', 12),
('RTM LMP_GHG_PRC', 1),
('RTM LMP_GHG_PRC', 7),
'DAM LMP_ENE_PRC',
'DAM LMP_LOSS_PRC',
'Actual_Demand',
('RTM LMP_PRC', 1),
'DAM LMP_PRC',]]
          
Y = df[Y_cols]
X = df[X_cols]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10615, shuffle=False, random_state=42)

# ss1 = MinMaxScaler(feature_range=(0, 1))
# X_train = ss1.fit_transform(X_train)
# X_test = ss1.transform(X_test)
# ss2 = MinMaxScaler(feature_range=(0, 1))
# Y_train = ss2.fit_transform(Y_train)
# Y_test = ss2.transform(Y_test)



regr_multirf = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False)

In [8]:
regr_multirf.fit(X_train, Y_train)

C:\Users\Nika\Anaconda3\envs\PricePrediction\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)